<a href="https://colab.research.google.com/github/SomaOkuda/Tweet-Analysis/blob/main/japanese_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install janome
#MeCabインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.996.3
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
!pip install emoji
!pip install japanize-matplotlib
!pip install gensim==3.4.0
!pip install smart_open==1.9.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import smart_open
import gensim
from gensim.models import word2vec
import glob
import pickle
import emoji


import MeCab
tagger=MeCab.Tagger('-Owakati')

from janome.tokenizer import Tokenizer
import time
# Tokenizerインスタンスの生成 
t = Tokenizer()

In [ ]:
csv_files1 = glob.glob('/content/drive/My Drive/Tweet/tweet/auto_test/*.csv') #jpサンプルツイートフォルダ内結合
data_list1 = []
for file in csv_files1:
  data_list1.append(pd.read_csv(file, lineterminator='\n'))  
df = pd.concat(data_list1, axis=0, sort=False)
df = df.drop_duplicates(keep='last') #重複の削除、列の最後の方が時刻が遅いため、lastで指定
df=df.reset_index(drop=True)

elim=[]
for i in range(len(df)):
  if str(df.iloc[i]['text']).startswith('RT') is True:
    elim.append(i)

#RT除去
df1=df.drop(elim)
df1=df1.reset_index(drop=True)
len(df1)

In [ ]:
elim_http=[]
for j in range(len(df1)):
  user_text=df1.iloc[j]['text']
  word_list=list(tagger.parse(user_text).split())
  if 'https' in word_list:
    elim_http.append(j)
  elif 'http' in word_list:
    elim_http.append(j)
  elif '@' in word_list:
    elim_http.append(j)

#http除去
df2=df1.drop(elim_http)
df2=df2.reset_index(drop=True)
len(df2) 

In [ ]:
def Remove_word(list):
  rems = ['@', ',', '，', '。', '', ' ', 'http', 'https','//', '、', '×', '.',
            '+', '＋', '/',', ', '(', ')', '（', '）', '://', 'RT', ':', ';','：', '；',
            '」','「', '＠', '-', '／',  '%', '％', '!', '！', '#', '＃',
            '$', '＄', '=', '＝', 'ー', '￥', '^', '{', '}', '~', '～', 
            '&', '＆', '・', '?', '??', '？', '？？',  '<', '＜', '>', '＞', '_', '＿', 
            'co', 'jp', '(@', '○', '『', '』', '”', '’', '"', '…', '‼', '(+'
            ')」', '〇', '【', '】', '[', ']', '⁉', 'gt',
            'から', 'より', 'こそ', 'でも', 'しか','さえ', 'けれど', 'たり', 'つつ', 'とも',
            'たら', 'ある', 'なら', 'のに', 'です', 'ます', 'する', 'ほど', 'ない', 'くる',
            'なり', 'そう', 'まし', 'その', 'この', 'あの', 'せる', 'どう', 'ため', 'どこ',
            'いる', 'これ', 'それ', 'あれ', 'いい', 'など', 'あっ', 'もう', 'さん', 'じゃ',
            'から', 'あり', 'ので', 'とも', 'ませ', 'でし', 'とき', 'こと', 'なる', 'って',
            'ただ', 'まで', 'もの', 'つか', 'なっ', 'でき', 'もっ', 'けど', 'ほぼ', 'なー',
            'そこ', 'ここ', 'だろ', 'なん', 'だっ', 'なあ', 'っけ', 'せる', 'やっ', 'また',
            'どれ', 'なれ', 'かも', 'いく', 'いけ', 'いう', 'たい', 'あと', 'かも', 'しれ',  
            'こう', 'なく', 'よく', 'だけ', 'れる', 'よう', 'かけ', 'どの', 'てる', 'とか',
            'という', '思う', '思っ', 'として', 'ところ', 'しまう', 'なんか', 'そんな', 
            'でしょ', 'しまい', 'わかり', 'まま'
            '\u200d', '\u3000#'
            ]
  for number in range(10000):
    rems.append(str(number))

# 一文字のひらがなとカタカナ，全角数字，アルファベット大文字・小文字
  hirakana    = [chr(i) for i in range(12353, 12436)]
  katakana    = [chr(i) for i in range(12449, 12533)]
  zenkaku_num = [chr(i) for i in range(65296, 65296+10)]
  alph_L      = [chr(i) for i in range(97, 97+26)]
  alph_S      = [chr(i) for i in range(65, 65+26)]

  for a in range(len(hirakana)):
    rems.append(hirakana[a])
  for b in range(len(katakana)):
    rems.append(katakana[b])
  for c in range(len(zenkaku_num)):
    rems.append(zenkaku_num[c])
  for d in range(len(alph_L)):
    rems.append(alph_L[d])
  for e in range(len(alph_S)):
    rems.append(alph_S[e])

  words = [x for x in list if (x not in rems)]
  return words

In [ ]:
# テキストを引数として、形態素解析の結果、名詞・動詞・形容詞(原形)のみを配列で抽出する関数を定義 
def extract_word(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞','形容詞']]


all_users_words=[]
df3=df2.sample(frac=0.5) #ツイート数が大きいから50%だけ抽出してる
df3=df3.reset_index(drop=True)

for i in range(len(df3)):
  user_text=df3.iloc[i]['text']
  #text=list(tagger.parse(user_text).split())
  words=extract_word(user_text)
  all_users_words.append(words)
  
all_users_words1=[e for inner_list in all_users_words for e in inner_list]

In [ ]:
import pickle
f = open('/content/drive/My Drive/Tweet/w2b/210110_vaccine.txt', 'wb')
list_row = all_users_words1
pickle.dump(list_row, f)

BoW構築

In [ ]:
def id_preprocess(words, word_to_id, id_to_word):
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    return word_to_id, id_to_word

def corpus_preprocess(words, word_to_id, id_to_word):
    corpus = np.array([word_to_id[w] for w in words])
    return corpus

word_to_id = {}
id_to_word = {}
corpus_dic = {}

word_to_id, id_to_word = id_preprocess(all_users_words1, word_to_id, id_to_word)
corpus_dic = corpus_preprocess(all_users_words1, word_to_id, id_to_word)

bag_of_words = {}
vocab_size = len(word_to_id) # = len(id_to_word) 

bag_of_words = {}
corpus_size = len(corpus_dic)
  
for i in range(vocab_size):
  count = 0
  for j in range(corpus_size):
    try:
      if corpus_dic[j] == i: # 元の文書を走査していき，一致していればカウントする。
        count += 1
      else:
        pass
    except:
      pass
  bag_of_words[id_to_word[i]] = count
  
bag_of_words = sorted(bag_of_words.items(), key = lambda x: x[1], reverse=True)

In [ ]:
df_bow = pd.DataFrame(bag_of_words)
df_bow.to_csv('/content/drive/MyDrive/Tweet/w2b/220110_including_vaccine.csv', index=False)

*Twitterワクチンを含むツイートのWord2vec構築*


In [ ]:
import pandas as pd

Table=pd.read_csv('/content/drive/MyDrive/Tweet/w2b/220110_including_vaccine.csv')
word_df=Table[Table["1"]>=5]
word_df=word_df.reset_index(drop=True)
len(word_df)
#words_exist=word_df["0"].values.tolist()
#print(words_exist[:5])

In [ ]:
from janome.tokenizer import Tokenizer
import time

# Tokenizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞・形容詞(原形)のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞', '形容詞']]

# 全体のテキストを句点('。')で区切った配列にする。 
sent = list(df3["text"])
# それぞれの文章を単語リストに変換(処理に数分かかります)

time1=time.time()
word_list=[]
for sentence in sent:
  word_list.append(extract_words(sentence))

time2=time.time()
total_time=time2-time1
print(total_time)
#estimate:40分

In [ ]:
model = word2vec.Word2Vec(word_list,
                          size=50,  # ベクトルの次元
                          min_count=3,  # 最小3回以上出ている単語のみの使用
                          window=5,  # window幅
                          iter=100  # 学習の繰り返し数
                         )

model.save("testmodel")

cos類似度探索＊単語の類似度を測る

In [ ]:
#model= word2vec.Word2Vec.load("testmodel_1")
ret = model.wv.most_similar(positive=['XX']) 
for item in ret:
    print(item[0], item[1])
#絵文字含まれない

In [ ]:
def cal_cos_similarity(q, d):
  return np.dot(q, d) / (np.linalg.norm(q) * np.linalg.norm(d))

print(cal_cos_similarity(model.wv['XX'], model.wv['XX']))

In [ ]:
total_cos_list=[]
words_exist=word_df["0"].values.tolist()
for word in words_exist:
  cos_list=[]
  for word2 in words_exist:
    cos_list.append(cal_cos_similarity(model.wv[word], model.wv[word2]))
  total_cos_list.append(cos_list)

In [ ]:
df_cos_similarity=pd.DataFrame({'name': words_exist,
                                'similarity': total_cos_list})

splitted = df_cos_similarity['similarity'].apply(pd.Series)
splitted.columns = words_exist
df_cos_similarity=pd.concat([df_cos_similarity['name'], splitted], axis=1)

In [ ]:
df_cos_similarity.to_csv('cos_similarity.csv')

In [ ]:
print(df_cos_similarity)

In [ ]:
#df1=df.drop('Unnamed: 0', axis=1)
df_cos_similarity1=df_cos_similarity.set_index('name')
#df2=df1.sample(frac=1)
print(df_cos_similarity1)

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram
import pandas as pd
import matplotlib as plt
import japanize_matplotlib #日本語化matplotlib
import seaborn as sns
sns.set(font="IPAexGothic") #日本語フォント設定

result1 = linkage(df_cos_similarity1, 
                  metric = 'braycurtis', 
                  #metric = 'canberra', 
                  #metric = 'chebyshev', 
                  #metric = 'cityblock', 
                  #metric = 'correlation', 
                  #metric = 'cosine', 
                  #metric = 'euclidean', 
                  #metric = 'hamming', 
                  #metric = 'jaccard', 
                  #method= 'single')
                  method = 'average')
                  #method= 'complete')
                  #method='weighted')

In [ ]:
# 指定したクラスタ数でクラスタを得る関数を作る。
def get_cluster_by_number(result, number):
    output_clusters = []
    x_result, y_result = result.shape
    n_clusters = x_result + 1
    cluster_id = x_result + 1
    father_of = {}
    df1 = pd.DataFrame(result)
    x1 = []
    y1 = []
    x2 = []
    y2 = []
    for i in df1.index:
        n1 = int(df1.iloc[i][0])
        n2 = int(df1.iloc[i][1])
        val = df1.iloc[i][2]
        n_clusters -= 1
        if n_clusters >= number:
            father_of[n1] = cluster_id
            father_of[n2] = cluster_id

        cluster_id += 1

    cluster_dict = {}
    for n in range(x_result + 1):
        if not n in father_of:
            output_clusters.append([n])
            continue

        n2 = n
        m = False
        while n2 in father_of:
            m = father_of[n2]
            #print [n2, m]
            n2 = m

        if not m in cluster_dict:
            cluster_dict.update({m:[]})
        cluster_dict[m].append(n)

    output_clusters += cluster_dict.values()

    output_cluster_id = 0
    output_cluster_ids = [0] * (x_result + 1)
    for cluster in sorted(output_clusters):
        for i in cluster:
            output_cluster_ids[i] = output_cluster_id
        output_cluster_id += 1

    return output_cluster_ids

In [ ]:
cluster_n=50
cluster=get_cluster_by_number(result1, cluster_n)
df_cluster=pd.DataFrame({'name': df_cos_similarity.name, 
                         'cluster': cluster})
df_cluster_dict=df_cluster.set_index('name').to_dict(orient='dict')
df_cluster_dict=df_cluster_dict['cluster']

In [ ]:
word_list = []
import csv
for i in range(50):
  df_i=df_cluster[df_cluster['cluster']==i]
  print(i)
  print(list(df_i.name)[:20])
  #word_list.append(list(df_i.name)[:50])
#with open('/content/drive/My Drive/Tweet/w2b/vaccine.csv', 'w') as f:
     # write = csv.writer(f) 
    #  write.writerow("words")
   #   write.writerows(word_list)

In [ ]:
f = open("/content/drive/My Drive/Tweet/w2b/vaccine.txt","rb")
all_users_BoW = pickle.load(f)

all_freq=[]
for bow in all_users_BoW:
  Table = pd.DataFrame(bow)
  cluster_n=[]
  for i in range(len(Table)):
    if Table.iloc[i][0] in df_cluster_dict:
      cluster_n.append(df_cluster_dict[Table.iloc[i][0]])
    else:
      cluster_n.append('error')
  Table['cluster_n']=pd.Series(cluster_n)

  freq=[]
  freq1=[]
  for i in range(200):
    df_c=Table[Table['cluster_n']==i]
    df_c=df_c.reset_index(drop=True)
    freq.append(df_c[1].sum())
  for i in range(len(freq)):
    freq1.append(freq[i]*100/sum(freq))
  all_freq.append(freq1)

In [ ]:
df_pred=pd.read_excel('/content/drive/MyDrive/Income_prediction/0308/0317_cluster_default.xlsx')
c=pd.DataFrame(all_freq)
df_pred=pd.concat([df_pred, c], axis=1)
df_pred.to_excel('/content/drive/MyDrive/Income_prediction/0324_jp_twitter_w2v/0406_cluster_200.xlsx')